In [2]:
#Import Libraries

import sqlite3
import numpy as np 
import pandas as pd 
import pandasql as ps
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression

In [3]:
#Load data into DataFrames

df_sales = pd.read_csv('../src/data/EXTR_RPSale.csv')
df_res = pd.read_csv('../src/data/EXTR_ResBldg.csv')
df_lookup = pd.read_csv('../src/data/EXTR_LookUp.csv')
df_parcel = pd.read_csv('../src/data/EXTR_Parcel.csv', encoding='latin-1')

/Users/mihirbhagat/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (1,2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/mihirbhagat/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
#Create 'Year' and 'MajorMinor' columns in df_sales

df_sales['year'] = pd.DatetimeIndex(df_sales['DocumentDate']).year
df_sales['Major'] = df_sales['Major'].astype(str)
df_sales['Minor'] = df_sales['Minor'].astype(str)
df_sales['MajorMinor'] = df_sales['Major'] + '-' + df_sales['Minor']

In [5]:
#Filter for 2019 

df_sales_19 = df_sales[df_sales['year']==2019]

In [6]:
#Filter for PropertyType and SalePrice 

q1 = ("""SELECT * FROM df_sales_19
WHERE (PropertyType = 11 or PropertyType = 12 or PropertyType = 13 or PropertyType = 14) and SalePrice > 0
""")

In [7]:
df_sales_19 = ps.sqldf(q1)

In [8]:
#Create'MajorMinor' columns in df_res

df_res['Major'] = df_res['Major'].astype(str)
df_res['Minor'] = df_res['Minor'].astype(str)
df_res['MajorMinor'] = df_res['Major'] + '-' + df_res['Minor']

In [9]:
#Create'MajorMinor' columns in df_parcel

df_parcel['Major'] = df_parcel['Major'].astype(str)
df_parcel['Minor'] = df_parcel['Minor'].astype(str)
df_parcel['MajorMinor'] = df_parcel['Major'] + '-' + df_parcel['Minor']

In [10]:
#Merge Sales and Parcel 

new_df = pd.merge(df_sales_19, df_parcel, how = 'left',on =['MajorMinor'])

In [11]:
#Merge the merged DF with Res 

final_df = pd.merge(new_df, df_res, how = 'left',on =['MajorMinor'])

In [12]:
#Create a waterfront column 

final_df['is_waterfront'] = final_df['WfntLocation'] > 0

In [13]:
#How many waterfront in our DF 

final_df['WfntLocation'].value_counts()

0.0    19586
8.0       99
6.0       93
9.0       56
3.0       55
7.0       28
5.0        3
1.0        2
4.0        1
Name: WfntLocation, dtype: int64

In [14]:
final_df.shape

(22931, 158)

In [15]:
final_df.describe()

,ExciseTaxNbr,SalePrice,PropertyType,PrincipalUse,SaleInstrument,SaleReason,PropertyClass,year,Range,Township,...,FpMultiStory,FpFreestanding,FpAdditional,YrBuilt,YrRenovated,PcntComplete,Obsolescence,PcntNetCondition,Condition,AddnlCost
count,2.293100e+04,2.293100e+04,22931.000000,22931.000000,22931.00000,22931.000000,22931.000000,22931.0,19923.000000,19923.000000,...,19375.000000,19375.000000,19375.000000,19375.000000,19375.000000,19375.000000,19375.000000,19375.000000,19375.000000,19375.000000
mean,3.000581e+06,8.415076e+05,11.338755,5.585278,3.10331,1.002965,7.389124,2019.0,4.588516,23.823571,...,0.303639,0.067406,0.175639,1974.554219,95.131200,0.597677,0.123716,0.006761,3.434942,781.840206
std,1.688221e+04,2.315672e+06,0.933239,1.243877,1.39005,0.224512,1.651478,0.0,1.133155,1.669224,...,0.501299,0.258835,0.399190,30.522600,425.617303,6.852864,2.517477,0.433221,0.679666,2683.674905
min,2.968844e+06,1.000000e+01,11.000000,0.000000,2.00000,1.000000,0.000000,2019.0,2.000000,19.000000,...,0.000000,0.000000,0.000000,1900.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
25%,2.986058e+06,4.395000e+05,11.000000,6.000000,3.00000,1.000000,8.000000,2019.0,4.000000,23.000000,...,0.000000,0.000000,0.000000,1954.000000,0.000000,0.000000,0.000000,0.000000,3.000000,0.000000
50%,3.001453e+06,6.300000e+05,11.000000,6.000000,3.00000,1.000000,8.000000,2019.0,4.000000,24.000000,...,0.000000,0.000000,0.000000,1978.000000,0.000000,0.000000,0.000000,0.000000,3.000000,0.000000
75%,3.014598e+06,8.850000e+05,11.000000,6.000000,3.00000,1.000000,8.000000,2019.0,5.000000,25.000000,...,1.000000,0.000000,0.000000,2001.000000,0.000000,0.000000,0.000000,0.000000,4.000000,0.000000
max,3.035196e+06,1.630510e+08,14.000000,7.000000,27.00000,18.000000,9.000000,2019.0,13.000000,26.000000,...,5.000000,2.000000,3.000000,2019.000000,2019.000000,100.000000,99.000000,40.000000,5.000000,60000.000000


In [16]:
final_df['SqFtProp'] = final_df['SqFtTotLiving']/final_df['SqFtLot']

In [17]:
final_df['SqFtProp'].describe()

count    19375.000000
mean         0.345664
std          0.311609
min          0.000565
25%          0.157588
50%          0.254000
75%          0.428726
max          4.509395
Name: SqFtProp, dtype: float64

In [18]:
final_df['CostSqFt'] = final_df['SalePrice']/final_df['SqFtTotLiving']

In [19]:
final_df['CostSqFt'].describe()

count    19375.000000
mean       393.728628
std        364.712540
min          0.002268
25%        258.123872
50%        350.318471
75%        466.145139
max      29629.629630
Name: CostSqFt, dtype: float64

In [20]:
fsm_df_3 = final_df[["SalePrice", "BldgGrade"]].copy()
fsm_df_3.dropna(inplace=True)
fsm_3 = ols(formula="SalePrice ~ BldgGrade", data=fsm_df_3).fit()
fsm_3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.337
Model:                            OLS   Adj. R-squared:                  0.337
Method:                 Least Squares   F-statistic:                     9865.
Date:                Thu, 05 Mar 2020   Prob (F-statistic):               0.00
Time:                        14:28:56   Log-Likelihood:            -2.8173e+05
No. Observations:               19375   AIC:                         5.635e+05
Df Residuals:                   19373   BIC:                         5.635e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -1.512e+06   2.36e+04    -64.186      0.000   -1.56e+06   -1.47e+06
BldgGrade   3.006e+05   3026.963     99.321      0.000    2.95e+05    3.07e+05
==============================================================================
Omnibus:                    28545.481   Durbin-Watson:                   1.787
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         26032063.399
Skew:                           8.655   Prob(JB):                         0.00
Kurtosis:                     181.736   Cond. No.                         51.9
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [21]:
fsm_df = final_df[["SalePrice", "SqFtTotLiving", "SqFtProp", "CostSqFt"]].copy()
fsm_df.dropna(inplace=True)
fsm = ols(formula="SalePrice ~ SqFtTotLiving + SqFtProp + CostSqFt", data=fsm_df).fit()
fsm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.663
Model:                            OLS   Adj. R-squared:                  0.663
Method:                 Least Squares   F-statistic:                 1.271e+04
Date:                Thu, 05 Mar 2020   Prob (F-statistic):               0.00
Time:                        14:28:56   Log-Likelihood:            -2.7518e+05
No. Observations:               19375   AIC:                         5.504e+05
Df Residuals:                   19371   BIC:                         5.504e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept     -4.812e+05   7492.040    -64.230      0.000   -4.96e+05   -4.67e+05
SqFtTotLiving   409.3854      2.642    154.932      0.000     404.206     414.565
SqFtProp       8.168e+04   8233.152      9.921      0.000    6.55e+04    9.78e+04
CostSqFt        955.7403      7.077    135.050      0.000     941.869     969.612
==============================================================================
Omnibus:                    23472.225   Durbin-Watson:                   2.116
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         68231497.856
Skew:                           5.405   Prob(JB):                         0.00
Kurtosis:                     293.520   Cond. No.                     8.47e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 8.47e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [22]:
df_res.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 514509 entries, 0 to 514508
Data columns (total 51 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Major               514509 non-null  object 
 1   Minor               514509 non-null  object 
 2   BldgNbr             514509 non-null  int64  
 3   NbrLivingUnits      514509 non-null  int64  
 4   Address             514509 non-null  object 
 5   BuildingNumber      514509 non-null  object 
 6   Fraction            514509 non-null  object 
 7   DirectionPrefix     514011 non-null  object 
 8   StreetName          514509 non-null  object 
 9   StreetType          514509 non-null  object 
 10  DirectionSuffix     514011 non-null  object 
 11  ZipCode             469267 non-null  object 
 12  Stories             514509 non-null  float64
 13  BldgGrade           514509 non-null  int64  
 14  BldgGradeVar        514509 non-null  int64  
 15  SqFt1stFloor        514509 non-nul

In [23]:
df_parcel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614337 entries, 0 to 614336
Data columns (total 82 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   Major                   614337 non-null  object 
 1   Minor                   614337 non-null  object 
 2   PropName                601485 non-null  object 
 3   PlatName                520002 non-null  object 
 4   PlatLot                 614337 non-null  object 
 5   PlatBlock               614337 non-null  object 
 6   Range                   614337 non-null  int64  
 7   Township                614337 non-null  int64  
 8   Section                 614337 non-null  int64  
 9   QuarterSection          614337 non-null  object 
 10  PropType                614337 non-null  object 
 11  Area                    614307 non-null  float64
 12  SubArea                 614307 non-null  float64
 13  SpecArea                17333 non-null   float64
 14  SpecSubArea         

In [24]:
df_145 = final_df[["SalePrice", "SqFtTotLiving", "SqFtProp", "CostSqFt"]].copy()
model_145 = ols(formula="SalePrice ~ SqFtTotLiving + SqFtProp + CostSqFt", data=df_145).fit()
model_145.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.663
Model:                            OLS   Adj. R-squared:                  0.663
Method:                 Least Squares   F-statistic:                 1.271e+04
Date:                Thu, 05 Mar 2020   Prob (F-statistic):               0.00
Time:                        14:28:57   Log-Likelihood:            -2.7518e+05
No. Observations:               19375   AIC:                         5.504e+05
Df Residuals:                   19371   BIC:                         5.504e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept     -4.812e+05   7492.040    -64.230      0.000   -4.96e+05   -4.67e+05
SqFtTotLiving   409.3854      2.642    154.932      0.000     404.206     414.565
SqFtProp       8.168e+04   8233.152      9.921      0.000    6.55e+04    9.78e+04
CostSqFt        955.7403      7.077    135.050      0.000     941.869     969.612
==============================================================================
Omnibus:                    23472.225   Durbin-Watson:                   2.116
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         68231497.856
Skew:                           5.405   Prob(JB):                         0.00
Kurtosis:                     293.520   Cond. No.                     8.47e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 8.47e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [25]:
#Combo 145 

df_145 = final_df[["SalePrice", "SqFtTotLiving", "SqFtProp", "CostSqFt"]].copy()
model_145 = ols(formula="SalePrice ~ SqFtTotLiving + SqFtProp + CostSqFt", data=df_145).fit()
model_145.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.663
Model:                            OLS   Adj. R-squared:                  0.663
Method:                 Least Squares   F-statistic:                 1.271e+04
Date:                Thu, 05 Mar 2020   Prob (F-statistic):               0.00
Time:                        14:28:57   Log-Likelihood:            -2.7518e+05
No. Observations:               19375   AIC:                         5.504e+05
Df Residuals:                   19371   BIC:                         5.504e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept     -4.812e+05   7492.040    -64.230      0.000   -4.96e+05   -4.67e+05
SqFtTotLiving   409.3854      2.642    154.932      0.000     404.206     414.565
SqFtProp       8.168e+04   8233.152      9.921      0.000    6.55e+04    9.78e+04
CostSqFt        955.7403      7.077    135.050      0.000     941.869     969.612
==============================================================================
Omnibus:                    23472.225   Durbin-Watson:                   2.116
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         68231497.856
Skew:                           5.405   Prob(JB):                         0.00
Kurtosis:                     293.520   Cond. No.                     8.47e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 8.47e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [26]:
final_df['TrafficNoise'].value_counts()

0.0    17160
1.0     1654
2.0      969
3.0      140
Name: TrafficNoise, dtype: int64

In [27]:
#0 is None
#1 is Moderate
#2 is High
#3 is Extreme 

In [28]:
#Query for Traffic

qt = ("""
SELECT SalePrice,
CASE
    WHEN TrafficNoise > 0 THEN 1
    ELSE 0
END AS traffic
FROM final_df
""")
traffic_df = ps.sqldf(qt)

In [29]:
#Add Traffic to final_df

final_df['Traffic'] = traffic_df['traffic']

In [ ]:
#Query for Power Lines

qpl = ("""
SELECT SalePrice,
CASE
    WHEN PowerLines == 'Y' THEN 1
    ELSE 0
END AS power_lines
FROM final_df
""")
power_lines_df = ps.sqldf(qpl)

In [ ]:
#Add PowerLines to final_df

final_df['PowerLines'] = power_lines_df['power_lines']

In [ ]:
#Query for Other Nuisance

qon = ("""
SELECT SalePrice,
CASE
    WHEN OtherNuisances == 'Y' THEN 1
    ELSE 0
END AS other_nuisance
FROM final_df
""")
other_nuisance_df = ps.sqldf(qon)

In [ ]:
#Add OtherNuisance to final_df

final_df['Other_Nuisance'] = other_nuisance_df['other_nuisance']

In [ ]:
#Query Any Nuisance 

qn = ("""
SELECT SalePrice,
CASE
    WHEN Other_Nuisance > 0 OR PowerLines > 0 OR Traffic > 0 THEN 1
    ELSE 0
END AS any_nuisance
FROM final_df
""")
any_nuisance_df = ps.sqldf(qn)

In [ ]:
#Add Any Nuisance to final_df

final_df['Any_Nuisance'] = any_nuisance_df['any_nuisance']

In [ ]:
#Check Values 

final_df['Any_Nuisance'].value_counts()

In [ ]:
#Combo 146

df_146 = final_df[["SalePrice", "SqFtTotLiving", "SqFtProp", "Any_Nuisance"]].copy()
model_146 = ols(formula="SalePrice ~ SqFtTotLiving + SqFtProp + Any_Nuisance", data=df_146).fit()
model_146.summary()

In [ ]:
#Combo 156

df_156 = final_df[["SalePrice", "SqFtTotLiving", "CostSqFt", "Any_Nuisance"]].copy()
model_156 = ols(formula="SalePrice ~ SqFtTotLiving + CostSqFt + Any_Nuisance", data=df_156).fit()
model_156.summary()

In [ ]:
#Query for Waterfront

qwater = ("""
SELECT SalePrice,
CASE
    WHEN WfntLocation > 0 THEN 1
    ELSE 0
END as waterfront
FROM final_df
""")
wfnt_df = ps.sqldf(qwater)

In [ ]:
#Add Any Waterfront to final_df

final_df['Any_Waterfront'] = wfnt_df['waterfront']

In [ ]:
#Query for Porch

qporch = ("""
SELECT SalePrice,
CASE
    WHEN SqFtOpenPorch > 0 OR SqFtEnclosedPorch > 0 OR SqFtDeck > 0 THEN 1
    ELSE 0
END AS has_porch
FROM final_df
""")
porch_df = ps.sqldf(qporch)

In [ ]:
#Add Any Porch to final_df

final_df['Any_Porch'] = porch_df['has_porch']

In [ ]:
#Combo 234

df_234 = final_df[["SalePrice", "Any_Porch", "Any_Waterfront", "SqFtProp"]].copy()
model_234 = ols(formula="SalePrice ~ Any_Porch + Any_Waterfront + SqFtProp", data=df_234).fit()
model_234.summary()

In [ ]:
#Combo 235

df_235 = final_df[["SalePrice", "Any_Porch", "Any_Waterfront", "CostSqFt"]].copy()
model_235 = ols(formula="SalePrice ~ Any_Porch + Any_Waterfront + CostSqFt", data=df_235).fit()
model_235.summary()

In [ ]:
#Combo 236

df_236 = final_df[["SalePrice", "Any_Porch", "Any_Waterfront", "Any_Nuisance"]].copy()
model_236 = ols(formula="SalePrice ~ Any_Porch + Any_Waterfront + Any_Nuisance", data=df_236).fit()
model_236.summary()

In [ ]:
#Combo 245

df_236 = final_df[["SalePrice", "Any_Porch", "SqFtProp", "CostSqFt"]].copy()
model_236 = ols(formula="SalePrice ~ Any_Porch + SqFtProp + CostSqFt", data=df_236).fit()
model_236.summary()

In [ ]:
final_df['BldgGrade'].describe()

In [ ]:
df_year = final_df[["SalePrice", "BldgGrade", "SqFtTotLiving"]].copy()
model_year = ols(formula="SalePrice ~ BldgGrade + SqFtTotLiving", data=df_year).fit()
model_year.summary()

In [ ]:
final_df['Condition'].describe()

In [60]:
df_parcel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614337 entries, 0 to 614336
Data columns (total 82 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   Major                   614337 non-null  object 
 1   Minor                   614337 non-null  object 
 2   PropName                601485 non-null  object 
 3   PlatName                520002 non-null  object 
 4   PlatLot                 614337 non-null  object 
 5   PlatBlock               614337 non-null  object 
 6   Range                   614337 non-null  int64  
 7   Township                614337 non-null  int64  
 8   Section                 614337 non-null  int64  
 9   QuarterSection          614337 non-null  object 
 10  PropType                614337 non-null  object 
 11  Area                    614307 non-null  float64
 12  SubArea                 614307 non-null  float64
 13  SpecArea                17333 non-null   float64
 14  SpecSubArea         

In [61]:
df_res.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 514509 entries, 0 to 514508
Data columns (total 51 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Major               514509 non-null  object 
 1   Minor               514509 non-null  object 
 2   BldgNbr             514509 non-null  int64  
 3   NbrLivingUnits      514509 non-null  int64  
 4   Address             514509 non-null  object 
 5   BuildingNumber      514509 non-null  object 
 6   Fraction            514509 non-null  object 
 7   DirectionPrefix     514011 non-null  object 
 8   StreetName          514509 non-null  object 
 9   StreetType          514509 non-null  object 
 10  DirectionSuffix     514011 non-null  object 
 11  ZipCode             469267 non-null  object 
 12  Stories             514509 non-null  float64
 13  BldgGrade           514509 non-null  int64  
 14  BldgGradeVar        514509 non-null  int64  
 15  SqFt1stFloor        514509 non-nul

In [ ]:
df_mihir_1 = final_df[["SalePrice", "YrBuilt"]].copy()
model_mihir_1 = ols(formula="SalePrice ~ YrBuilt", data=df_mihir_1).fit()
model_mihir_1.summary()

In [ ]:
df_mihir = final_df[["SalePrice", "BldgGrade", "SqFtTotLiving","Condition","YrRenovated","YrBuilt","SqFtProp","Bedrooms", "Any_Waterfront"]].copy()
model_mihir = ols(formula="SalePrice ~ BldgGrade + SqFtTotLiving + Condition + YrRenovated + YrBuilt + SqFtProp + Bedrooms + Any_Waterfront", data=df_mihir).fit()
model_mihir.summary()

In [ ]:
def rainbow(fsm):
    rainbow_statistic, rainbow_p_value = linear_rainbow(fsm)
    print("Rainbow statistic:", rainbow_statistic)
    print("Rainbow p-value:", rainbow_p_value)

In [ ]:
def overallmodfit(fsm,d):
    print("R^2: " ,fsm.rsquared) # R^2
    print("Adj. R^2: ",fsm.rsquared_adj) # Adj. R^2
    print("F-statistic: ",fsm.fvalue) # F-statistic
    print("Prob(F-statistic)/p-value: ",fsm.f_pvalue) # Prob(F-statistic)/p-value
    print('linearity')
    rainbow_statistic, rainbow_p_value = linear_rainbow(fsm)
    print("Rainbow statistic:", rainbow_statistic)
    print("Rainbow p-value:", rainbow_p_value)
    y = fsm_dfinal['SalePrice']
    y_hat = fsm_246.predict()
    print('homoscedasticity: Breusch Pagan test')
    lm, lm_p_value, fvalue, f_p_value = het_breuschpagan(y-y_hat, d)
    print("Lagrange Multiplier p-value:", lm_p_value)
    print("F-statistic p-value:", f_p_value)
    print('normality')
    jb, jbpv, skew, kurtosis = jarque_bera(y-y_hat)
    print('jarque_bera: ',jb)
    print('jarque_bera p-value: ',jbpv)

In [ ]:
overallmodfit(model_mihir,df_mihir)

In [ ]:
corr = df_final.corr()
mask = np.triu(np.ones_like(corr, dtype=np.bool))

fig1, ax1 = plt.subplots(figsize=(11, 9))
sns.heatmap(corr, mask=mask, ax=ax1)